- deploy via ssh
- recieve job from local via ssh
- obtain datainstances via globus
- execute job
- upload produced datainstances via globus
- return globus paths (indexes/keys?)

In [1]:
from pathlib import Path
from metasmith.agents.ssh import Agent

In [2]:
agent = Agent(
    ssh_command='ssh sockeye',
    ssh_address='sockeye',
    pre = f"""\
        module load gcc/9.4.0 apptainer/1.3.1
    """,
    home='~/metasmith',
    workspace='~/scratch/metasmith',
    globus_endpoint="64a5c402-05c4-4607-bbad-46a9c2aebd98"
)

# agent = Agent(
#     ssh_command='ssh cedar',
#     ssh_address='cedar',
#     pre = f"""\
#         module load StdEnv/2023 apptainer/1.3.5
#     """,
#     home='~/metasmith',
#     workspace='~/scratch/metasmith',
# )

# agent.Deploy()
# agent.Deploy(force=True)

In [3]:
from metasmith.models.libraries import *
from local.constants import WORKSPACE_ROOT
from local.utils import LinkifyPath
CACHE = WORKSPACE_ROOT/"main/local_mock/cache"

types = DataTypeLibrary.Load(WORKSPACE_ROOT/"main/local_mock/prototypes/metagenomics.dev3.yml")
len(types)

6

In [4]:
xgdb_path = CACHE/"test.xgdb"
refdb_path = CACHE/"ref.xgdb"
_xgdb = DataInstanceLibrary()
refdb = DataInstanceLibrary()
_xgdb.Add(
    name="contigs",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/example.fna"),
    type=types["contigs"],
)
refdb.Add(
    name="reference.uniprot_sprot",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/uniprot_sprot.dmnd"),
    type=types["protein_reference_diamond"],
)
refdb.Add(
    name="container.diamond",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/diamond.oci.uri"),
    type=types["oci_image_diamond"],
)
refdb.Add(
    name="container.pprodigal",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/pprodigal.oci.uri"),
    type=types["oci_image_pprodigal"],
)
_xgdb.Save(xgdb_path)
refdb.Save(refdb_path)
LinkifyPath(xgdb_path/_xgdb._index_name)
LinkifyPath(refdb_path/refdb._index_name)
xgdb = DataInstanceLibrary.Load(xgdb_path)

./../../main/local_mock/cache/test.xgdb/info.yml
./../../main/local_mock/cache/ref.xgdb/info.yml


In [5]:
xgdb_g_path = CACHE/"testg.xgdb.yml"
refdb_g_path = CACHE/"testg.refdb.yml"

if not xgdb_g_path.exists():
    xgdb_g = xgdb.Move(
        dest=GlobusSource.Parse("https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test3%2Fxgdb%2F").AsSource(),
    )
    xgdb_g.Save(xgdb_g_path)
if not refdb_g_path.exists():
    refdb_g = refdb.Move(
        dest=GlobusSource.Parse("https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test3%2Frefdb%2F").AsSource(),
    )
    refdb_g.Save(refdb_g_path)

xgdb_g = DataInstanceLibrary.Load(xgdb_g_path)
refdb_g = DataInstanceLibrary.Load(refdb_g_path)
LinkifyPath(xgdb_g_path)
LinkifyPath(refdb_g_path)

./../../main/local_mock/cache/testg.xgdb.yml
./../../main/local_mock/cache/testg.refdb.yml


In [6]:
GlobusSource.Parse("https://app.globus.org/file-manager?destination_id=64a5c402-05c4-4607-bbad-46a9c2aebd98&destination_path=%2Fhome%2Ftxyliu%2F").endpoint

'64a5c402-05c4-4607-bbad-46a9c2aebd98'

In [7]:
trlib = TransformInstanceLibrary.Load([
    Path("./transforms/simple_1"),
    # Path("./transforms/dupe_test"),
])
for k, v in trlib:
    print(k.name, type(v))

diamond.py <class 'metasmith.models.libraries.TransformInstance'>
pprodigal.py <class 'metasmith.models.libraries.TransformInstance'>


In [8]:
from metasmith.models.workflow import WorkflowPlan

plan = WorkflowPlan.Generate(
    given=[xgdb_g, refdb_g],
    transforms=trlib,
    targets=[
        types["orf_annotations"].WithLineage([
            types["contigs"],
            # xgdb["example.fna"].type,
        ]),
    ],
)

for step in plan.steps:
    print([f"{x.source}" for x in step.uses], [f"{x.source}" for x in step.produces], sep="->")
    LinkifyPath(step.transform._source.address)

["Source(address='globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/refdb/pprodigal.oci.msm_DaORQPNcOLWU.uri', type=SourceType.GLOBUS, _hash=102204026265152225396459869697273031746939003151309883073453404794116985136543)", "Source(address='globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/xgdb/example.msm_sbq72Lgs35k6.fna', type=SourceType.GLOBUS, _hash=84723774456271901842268332685540500057626299381193261955467606788476030347185)"]->["Source(address='orfs.faa', type=SourceType.DIRECT, _hash=101873239097198290733880342831050169978545220014649872182640029742000354522415)"]
./../../main/local_mock/transforms/simple_1/pprodigal.py
["Source(address='globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/refdb/diamond.oci.msm_eqLcckPvXFTB.uri', type=SourceType.GLOBUS, _hash=47647831126271685893919590576411455385101697691183148169154647140868721415188)", "Source(address='orfs.faa', type=SourceType.DIRECT, _hash=10187323909719829073388034

In [9]:
plan._key

'VLr5Nw'

In [10]:
from metasmith.coms.ipc import LiveShell
from metasmith.logging import Log
from tempfile import TemporaryDirectory

from metasmith.models.workflow import WorkflowTask

with open(WORKSPACE_ROOT/"secrets/slurm_account") as f:
    slurm_account = f.read().strip()

plan = plan
local_temp = Path("./cache/mock_temp/")
# with LiveShell() as shell, TemporaryDirectory(prefix="msm.") as local_temp:
with LiveShell() as sh_local, LiveShell() as sh_remote:
    local_temp = Path(local_temp)
    sh_local.RegisterOnOut(lambda x: Log.Info(f"{x}"))
    sh_local.RegisterOnErr(lambda x: Log.Error(f"{x}"))
    sh_remote.RegisterOnOut(lambda x: print(f"R| {x}"))
    sh_remote.RegisterOnErr(lambda x: print(f"R| {x}"))
    remote_cache = Path(f"/tmp/metasmith.{plan._key}")

    task_path = local_temp/f"plan.json"
    task = WorkflowTask(plan=plan, agent=agent, config=dict(
        nextflow=dict(
            slurm_account=slurm_account
        ),
    ))
    task.Save(task_path)

    tr_path = local_temp/f"transforms.lib"
    trlib.Gather(tr_path, overwrite=True)

    res = sh_remote.Exec(agent.ssh_command)
    errs = [e for e in res.err if e not in {"Pseudo-terminal will not be allocated because stdin is not a terminal."}]
    assert len(errs) == 0, errs
    sh_remote.Exec(agent.pre)
    sh_remote.Exec(f"[ -e {remote_cache} ] && rm -rf {remote_cache}")
    sh_remote.Exec(f"mkdir -p {remote_cache}")
    sh_remote.Exec(f"{agent.home}/relay/msm_relay start")

    sh_local.Exec(f"rsync -avcp {task_path} {agent.ssh_address}:{remote_cache/task_path.name}")
    sh_local.Exec(f"rsync -avcp --exclude=__pycache__ {tr_path}/ {agent.ssh_address}:{remote_cache/tr_path.name}")

    msm = agent.home/"msm"
    sh_remote.Exec(f'{msm} api stage_workflow -a plan_dir={remote_cache}') # this is only ok because /tmp is mounted
    sh_remote.Exec(f'{msm} api execute_workflow -a plan_dir=/ws/runs/{plan._key}')

    sh_remote.Exec(f"rm -rf {remote_cache}")
    # sh_remote.Exec(f"{agent.home}/relay/msm_relay stop")
    sh_remote.Exec("exit")

2025-02-21_11-52-06  | starting [3] transfers
2025-02-21_11-52-06  | executing [3] local transfers


2025-02-21_11-52-06  | attempted [3] transfers
R| Pseudo-terminal will not be allocated because stdin is not a terminal.
R| 2025-02-21_11-52-09 E| relay server already running in [/home/txyliu/metasmith/relay/connections]
2025-02-21_11-52-11  | sending incremental file list
2025-02-21_11-52-11  | plan.json
2025-02-21_11-52-11  | sent 7,786 bytes  received 35 bytes  5,214.00 bytes/sec
2025-02-21_11-52-11  | total size is 7,663  speedup is 0.98
2025-02-21_11-52-11  | sending incremental file list
2025-02-21_11-52-11  | created directory /tmp/metasmith.VLr5Nw/transforms.lib
2025-02-21_11-52-11  | ./
2025-02-21_11-52-11  | diamond.py
2025-02-21_11-52-11  | pprodigal.py
2025-02-21_11-52-11  | _type_libraries/
2025-02-21_11-52-11  | _type_libraries/_aliases.yml
2025-02-21_11-52-11  | _type_libraries/metagenomics.dev3.yml
2025-02-21_11-52-11  | sent 2,788 bytes  received 162 bytes  5,900.00 bytes/sec
2025-02-21_11-52-11  | total size is 2,336  speedup is 0.79
R| INFO:    squashfuse not found,

In [11]:
for s in task.plan.steps:
    print(s.transform_key)

DL05P
0RiTl


In [12]:
task

WorkflowTask(plan=WorkflowPlan(given=[DataInstance(source=Source(address='globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/xgdb/example.msm_sbq72Lgs35k6.fna', type=SourceType.GLOBUS, _hash=84723774456271901842268332685540500057626299381193261955467606788476030347185), type=<{"data":"DNA sequence","format":"FASTA"}:WrFmM>, _key='56yRzLGzCQes', _hash=19334972164257911230731478465347275442739001055883031028978266831408011409278), DataInstance(source=Source(address='globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/refdb/diamond.oci.msm_eqLcckPvXFTB.uri', type=SourceType.GLOBUS, _hash=47647831126271685893919590576411455385101697691183148169154647140868721415188), type=<{"data":"software container","format":"OCI","provides":["diamond"]}:VXp91>, _key='moz2z9kmeP0w', _hash=64286419791206899067609878188129191632963384648126594607047859740592213305182), DataInstance(source=Source(address='globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_te

In [13]:
# error here
# path incorrect
# loading type library [/ws/runs/WSFa2f/_metasmith/transforms.lib/_type_libraries/metagenomics.dev3.yml]

In [14]:
GlobusSource.Parse(task.plan.given[0].source.address)

globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/xgdb/example.msm_sbq72Lgs35k6.fna

In [15]:
"https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test1%2F"

'https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test1%2F'